In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import re


""" 네이버 랭킹 뉴스 긁어오기 """
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

d_list = []
start_data = 20210101
end_data = 20210116
for date_int in range(start_data, end_data):
    date = str(date_int)
    url = "https://news.naver.com/main/ranking/popularDay.nhn?date=" + date
    html = requests.get(url, headers=headers).text
    soup = BS(html, 'html.parser')
    ranking_total = soup.find_all(class_='rankingnews_box')

    for item in ranking_total:
        media = item.a.strong.text
        news = item.find_all(class_="list_content")
        for new in news:
            d = {}
            d['media'] = media
            d['src'] = "https://news.naver.com/" + new.a['href']
            d['title'] = new.a.text
            d['date'] = date
            d_list.append(d)
df = pd.DataFrame(d_list)

In [2]:
df

,media,src,title,date
0,경향신문,https://news.naver.com//main/ranking/read.nave...,"한국 수출, 역대 최고 ‘12월 성적표’",20210101
1,경향신문,https://news.naver.com//main/ranking/read.nave...,‘윤석열 형’에 등돌린 ‘박범계 아우’…법무장관 후보자의 과거 발언들,20210101
2,경향신문,https://news.naver.com//main/ranking/read.nave...,"태권도장 관장들 ""태권도장 최소 운영 허가하라""",20210101
3,경향신문,https://news.naver.com//main/ranking/read.nave...,교도소 한 곳서 1110여명 감염...코로나19 온상 된 교도소에 미국도 속수무책,20210101
4,경향신문,https://news.naver.com//main/ranking/read.nave...,"추미애, 서울동부구치소 코로나 집단감염에 ‘페이스북 사과’",20210101
...,...,...,...,...
5355,레이디경향,https://news.naver.com//main/ranking/read.nave...,[백인혜의 SNS 톡톡] 인간 본능 취향저격 트렌드 ‘할메니얼’,20210115
5356,레이디경향,https://news.naver.com//main/ranking/read.nave...,호텔업계 2021 키워드 ‘안전’···힐링은 '덤',20210115
5357,레이디경향,https://news.naver.com//main/ranking/read.nave...,"[임성용의 보약밥상] 굴, 어디에 좋을까?",20210115
5358,레이디경향,https://news.naver.com//main/ranking/read.nave...,"[주바리의 까칠한 味수다] '집콕 시대' 배달의 맛집, 배슐랭 가이드를 알려주마",20210115


In [3]:
""" 필요 없는 문자 제거 """
def clean_text(row):
    text = row['title']
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("E-mail제거 : " , text , "\n")
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("URL 제거 : ", text , "\n")
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("한글 자음 모음 제거 : ", text , "\n")
    pattern = '<[^>]*>'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("태그 제거 : " , text , "\n")
    pattern = r'\([^)]*\)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("괄호와 괄호안 글자 제거 :  " , text , "\n")
    pattern = '[^\w\s]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("특수기호 제거 : ", text , "\n" )
    pattern = '[^\w\s]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("필요없는 정보 제거 : ", text , "\n" )
    pattern = '["단독"]'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '["속보"]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("단독 속보 제거 : ", text , "\n" )
    text = text.strip()
    # print("양 끝 공백 제거 : ", text , "\n" )
    text = " ".join(text.split())
    # print("중간에 공백은 1개만 : ", text )
    return text


df['title_c'] = df.apply(clean_text, axis=1)
df

,media,src,title,date,title_c
0,경향신문,https://news.naver.com//main/ranking/read.nave...,"한국 수출, 역대 최고 ‘12월 성적표’",20210101,한국 수출 역대 최고 12월 성적표
1,경향신문,https://news.naver.com//main/ranking/read.nave...,‘윤석열 형’에 등돌린 ‘박범계 아우’…법무장관 후보자의 과거 발언들,20210101,윤석열 형에 등돌린 박범계 아우법무장관 후자의 과거 발언들
2,경향신문,https://news.naver.com//main/ranking/read.nave...,"태권도장 관장들 ""태권도장 최소 운영 허가하라""",20210101,태권도장 관장들 태권도장 최소 운영 허가하라
3,경향신문,https://news.naver.com//main/ranking/read.nave...,교도소 한 곳서 1110여명 감염...코로나19 온상 된 교도소에 미국도 속수무책,20210101,교도소 한 곳서 1110여명 감염코로나19 온상 된 교도소에 미국도 수무책
4,경향신문,https://news.naver.com//main/ranking/read.nave...,"추미애, 서울동부구치소 코로나 집단감염에 ‘페이스북 사과’",20210101,추미애 서울동부구치소 코로나 집감염에 페이스북 사과
...,...,...,...,...,...
5355,레이디경향,https://news.naver.com//main/ranking/read.nave...,[백인혜의 SNS 톡톡] 인간 본능 취향저격 트렌드 ‘할메니얼’,20210115,백인혜의 SNS 톡톡 인간 본능 취향저격 트렌드 할메니얼
5356,레이디경향,https://news.naver.com//main/ranking/read.nave...,호텔업계 2021 키워드 ‘안전’···힐링은 '덤',20210115,호텔업계 2021 키워드 안전힐링은 덤
5357,레이디경향,https://news.naver.com//main/ranking/read.nave...,"[임성용의 보약밥상] 굴, 어디에 좋을까?",20210115,임성용의 약밥상 굴 어디에 좋을까
5358,레이디경향,https://news.naver.com//main/ranking/read.nave...,"[주바리의 까칠한 味수다] '집콕 시대' 배달의 맛집, 배슐랭 가이드를 알려주마",20210115,주바리의 까칠한 味수다 집콕 시대 배달의 맛집 배슐랭 가이드를 알려주마


In [4]:
""" 키워드 추출 from title """
from konlpy.tag import Kkma
from konlpy.tag import Komoran
lst = []
kkma = Kkma()
komoran = Komoran()

for idx_line in range(len(df)):
    nouns_list = komoran.nouns(df['title_c'].loc[idx_line])
    nouns_list_c = [nouns for nouns in nouns_list if len(nouns) > 1]    # 한글자는 이상한게 많아서 2글자 이상
    lst.append(set(nouns_list_c))
    #df.loc[[idx_line], 'keyword'] = set(nouns_list_c)
df['keyword'] = lst
df = df[df['media'] != '코리아헤럴드']    # 코리아헤럴드는 영어 제목임


In [5]:
df.loc[[1111],['keyword']]

,keyword
1111,"{대출, 관장, 오픈, 헬스, 통장, 불복}"


In [6]:
df

,media,src,title,date,title_c,keyword
0,경향신문,https://news.naver.com//main/ranking/read.nave...,"한국 수출, 역대 최고 ‘12월 성적표’",20210101,한국 수출 역대 최고 12월 성적표,"{한국, 12월, 역대, 수출, 최고, 성적표}"
1,경향신문,https://news.naver.com//main/ranking/read.nave...,‘윤석열 형’에 등돌린 ‘박범계 아우’…법무장관 후보자의 과거 발언들,20210101,윤석열 형에 등돌린 박범계 아우법무장관 후자의 과거 발언들,"{법무, 윤석열, 돌리, 박범계, 발언, 과거, 장관, 아우, 후자}"
2,경향신문,https://news.naver.com//main/ranking/read.nave...,"태권도장 관장들 ""태권도장 최소 운영 허가하라""",20210101,태권도장 관장들 태권도장 최소 운영 허가하라,"{허가, 태권도장, 운영, 관장, 최소}"
3,경향신문,https://news.naver.com//main/ranking/read.nave...,교도소 한 곳서 1110여명 감염...코로나19 온상 된 교도소에 미국도 속수무책,20210101,교도소 한 곳서 1110여명 감염코로나19 온상 된 교도소에 미국도 수무책,"{여명, 코로나, 미국, 19, 온상, 무책, 교도소, 감염}"
4,경향신문,https://news.naver.com//main/ranking/read.nave...,"추미애, 서울동부구치소 코로나 집단감염에 ‘페이스북 사과’",20210101,추미애 서울동부구치소 코로나 집감염에 페이스북 사과,"{추미애, 코로나, 사과, 구치소, 서울, 감염, 페이스북, 동부}"
...,...,...,...,...,...,...
5355,레이디경향,https://news.naver.com//main/ranking/read.nave...,[백인혜의 SNS 톡톡] 인간 본능 취향저격 트렌드 ‘할메니얼’,20210115,백인혜의 SNS 톡톡 인간 본능 취향저격 트렌드 할메니얼,"{인간, 취향저격, 메니, 트렌드, 본능, 백인}"
5356,레이디경향,https://news.naver.com//main/ranking/read.nave...,호텔업계 2021 키워드 ‘안전’···힐링은 '덤',20210115,호텔업계 2021 키워드 안전힐링은 덤,"{업계, 키워드, 힐링, 호텔, 안전}"
5357,레이디경향,https://news.naver.com//main/ranking/read.nave...,"[임성용의 보약밥상] 굴, 어디에 좋을까?",20210115,임성용의 약밥상 굴 어디에 좋을까,"{약밥, 성용}"
5358,레이디경향,https://news.naver.com//main/ranking/read.nave...,"[주바리의 까칠한 味수다] '집콕 시대' 배달의 맛집, 배슐랭 가이드를 알려주마",20210115,주바리의 까칠한 味수다 집콕 시대 배달의 맛집 배슐랭 가이드를 알려주마,"{바리, 배달, 시대, 가이드, 맛집, 수다}"


In [24]:
a = tuple(sorted(list(df['keyword'][333])))
a

('여성', '이유', '폭력')

In [10]:
list(df['keyword'][0])

['한국', '12월', '역대', '수출', '최고', '성적표']

In [15]:
from collections import Counter

In [25]:
b = list(Counter(a).items())
b

[('여성', 1), ('이유', 1), ('폭력', 1)]

In [34]:
edge_list = []
for keywords_dict in df['keyword']:
    keywords = list(keywords_dict)
    num_keyword = len(keywords)
    
    if num_keyword > 0:
        for i in range(num_keyword-1):
            for j in range(i+1, num_keyword):
                edge_list += [tuple(sorted([keywords[i], keywords[j]]))]    # node 간 위해 sorted 사용
edges = list(Counter(edge_list).items())

In [29]:
edges

[(('12월', '한국'), 1),
 (('역대', '한국'), 1),
 (('수출', '한국'), 2),
 (('최고', '한국'), 1),
 (('성적표', '한국'), 1),
 (('12월', '역대'), 1),
 (('12월', '수출'), 1),
 (('12월', '최고'), 1),
 (('12월', '성적표'), 1),
 (('수출', '역대'), 2),
 (('역대', '최고'), 4),
 (('성적표', '역대'), 1),
 (('수출', '최고'), 1),
 (('성적표', '수출'), 1),
 (('성적표', '최고'), 1),
 (('법무', '윤석열'), 1),
 (('돌리', '법무'), 1),
 (('박범계', '법무'), 2),
 (('발언', '법무'), 1),
 (('과거', '법무'), 1),
 (('법무', '장관'), 3),
 (('법무', '아우'), 1),
 (('법무', '후자'), 2),
 (('돌리', '윤석열'), 1),
 (('박범계', '윤석열'), 4),
 (('발언', '윤석열'), 1),
 (('과거', '윤석열'), 1),
 (('윤석열', '장관'), 1),
 (('아우', '윤석열'), 1),
 (('윤석열', '후자'), 1),
 (('돌리', '박범계'), 1),
 (('돌리', '발언'), 1),
 (('과거', '돌리'), 1),
 (('돌리', '장관'), 1),
 (('돌리', '아우'), 1),
 (('돌리', '후자'), 1),
 (('박범계', '발언'), 1),
 (('과거', '박범계'), 2),
 (('박범계', '장관'), 5),
 (('박범계', '아우'), 1),
 (('박범계', '후자'), 3),
 (('과거', '발언'), 1),
 (('발언', '장관'), 1),
 (('발언', '아우'), 1),
 (('발언', '후자'), 1),
 (('과거', '장관'), 1),
 (('과거', '아우'), 1),
 (('과거', '후자'), 1),
 (('아우', '장관')

In [33]:
import networkx as nx

G = nx.Graph((x, y, {'weight':v}) for (x,y), v in edges)
G

In [47]:
""" Community 추출 """
import community
from community import community_louvain as cl

partition = cl.best_partition(G)
nx.set_node_attributes(G, partition, "community")   # graph G에 community 속성 추가


""" Gephi file 작성 """
nx.write_gexf(G, '202101_keyword_community.gexf')